# Week 3 Advanced Aggregation

In this activity, we look at 1) more sophisticated group-by operations and 2) data visualisation.

Load the titanic data (titanic.csv) into a pandas DataFrame called ‘titanic’ using Pandas and print out the first 5 rows:


In [22]:
import pandas as pd
titanic = pd.read_csv('titanic.csv')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 1. Data Aggregation

## 1.1 Multiple aggregation operations

Last week we learnt how to perform aggregation operations to compute the mean or sum on individual columns. We also learnt how to group the data according to certain attributes prior to performing the aggregation. 

Oftentimes we'd like to compute multiple aggregation operations at the same time. Here is an example where we compute statistics on multiple columns at once. (Note that you could also use the method to compute different aggregation functions on the same column of data.)

We specify the set of aggregation operations we wish to perform by detailing:

the columns the aggregation should be applied to (in this case the two columns are 'who' and 'age'), 
the name of the resulting new columns (we'll call them 'passengers' and 'average age'), and
the aggregation operations to apply in each case ('count' and 'mean'):

In [23]:
fun = {'who':'count','age':'mean'}

So now we've defined an aggregation operation that both counts the number of passengers and computes their average age. Let's apply that operation to the rows in the titanic table, grouping them by passenger 'class'. To apply the operation we use the 'agg()' function:



In [26]:
groupbyClass = titanic.groupby('class').agg(fun)
groupbyClass

,who,age
class,,
First,216,38.233441
Second,184,29.877630
Third,491,25.140620


In [27]:
groupbyClass = titanic.groupby('class').agg(fun)
# The next line is not complete, think about it and tell me why
groupbyClass.rename(
    columns={"who":"passengers", "age":"average age"},
    inplace = True
)
groupbyClass = groupbyClass.reset_index()
groupbyClass

,class,passengers,average age
0,First,216,38.233441
1,Second,184,29.877630
2,Third,491,25.140620


Alternatively

Have a look at the output, which has now been grouped by passenger class. Which class had the most passengers and which one had the oldest passengers on average?

**Practice 1a**: Modify the aggregation operation 'fun' above so that it also finds the age of the oldest and youngest passengers in each class. Note that all aggregate operations being applied to the same column need to be placed within the same set of curly braces '{}' and separated by commas ','. So fill in the MISSING parts of the function below:

**Solution**:

In [ ]:
# From question
# fun2 = {'who':'count','age':{'mean', [MISSING], [MISSING]}}
fun2 = {'who':'count', 'age':{'mean','max','min'}}

In [ ]:
groupbyClass2 = titanic.groupby('class').agg(fun2)
groupbyClass2 = groupbyClass2.rename({"who":"passengers"}, axis=1)
# It’s incomplete, add in the renaming and re-indexing here also
groupbyClass2

**Practice 1b**: So was the oldest passenger traveling in 'first', 'second' or 'third' class? 

**Solution**: First class

In order to turn the output of the groupby operation into a DataFrame that can be further manipulated, we need to "flatten it" using the 'reset_index()' and 'droplevel()' commands. Have a look at the outputs of the following commands one after the other (by printing out the table each time) to see what they produce. 

In [ ]:
groupbyClass2 = groupbyClass2.reset_index()
# turn 'class' groups into column values
groupbyClass2.columns = groupbyClass2.columns.droplevel(0)
# drop the top level in the column hierarchy
groupbyClass2

In [ ]:
#groupbyClass2 = groupbyClass2.rename({"count":"passengers","max":"max age", "min":"min age", "mean":"average age"}, axis = 1)
#groupbyClass2 = groupbyClass2.rename(columns={"count":"passengers","max":"max age", "min":"min age", "mean":"average age"})
groupbyClass2.rename(columns={"count":"passengers","max":"max age", "min":"min age", "mean":"average age"}, inplace = True)
groupbyClass2

Flattening caused us to lose the column name for the 'class' attribute. We can rename the column as follows:

In [ ]:
groupbyClass2.rename(columns = {'':'class'},inplace = True) # rename the first column to be 'class'
groupbyClass2

## 1.2 Custom aggregation operations

There are many inbuilt functions in Python that can be used to aggregate data over columns. For example the 'nunique' function will count the number of unique values in a list.

Sometimes the function we need isn't available, however, because what we are after is too specific. For example, if we have a list of values, we might wish to count only those elements in the list with value above a certain threshold. Using the 'for' syntax in Python we can write an expression to count the elements as follows:

In [ ]:
my_list = (80,20,64,19,56,12,88)
sum(e>50 for e in my_list)

The expression is checking for each element 'e' in 'my_list' whether the value is greater than 50 or not, the sum() function is then counting the number of times the greater-than expression returns TRUE (i.e. the value 1). 

Now that we have a piece of code that can count the number of values that fit a condition, we'd like to use it in an aggregation operation over a column of a DataFrame. We can do that using an anonymous function (called a lambda function) in Python. The syntax to create an anonymous function is to write 'lambda x:' followed by the function itself, where 'x' is the name of the variable that appears in the function:

In [ ]:
fun3 = {'age':{'nunique',lambda x: sum(e>50 for e in x)}}

So we have defined a new aggregation operation 'fun3', which will create two new columns, a 'unique age count' column that counts the number of distinct values in the 'age' column using the function 'nunique' and a 'over 50s count' column that counts the number of values in the 'age' column that are greater than 50.

Again, we can reformat the DataFrame so it's ready for use:

In [ ]:
groupbyClass3 = titanic.groupby('class').agg(fun3).reset_index()  # turn groups into column values
groupbyClass3.columns = groupbyClass3.columns.droplevel(0)        # drop the top level in column hierarchy
groupbyClass3.rename(
    columns = {
        '':'class',
        'nunique':'unique age count',
        '<lambda_0>':'over 50s count'},
    inplace = True
)
groupbyClass3                                                    # print out the table

**Practice 2**: Interpret the output and discuss your finding with other students.

## 2. Data Visualization

In order to plot data in Python, we use the 'matplotlib' library: 

In [ ]:
import matplotlib.pylab as plt

When using Python in a Jupyter Notebook, you need to add also the following 'magic line' to make sure that graphs are shown inline in the notebook.

In [ ]:
%matplotlib inline

### 2.1 Basic Plots

We'll continue to use titanic data set and let's call the 'plot' routine to have a look at the data:

In [ ]:
plt.plot(titanic.fare)
plt.show()

The figure looks a little complicated, but it is just plotting the fare for each passenger.

#### Practice 3: 
How many passengers were there in total?

**Solution**:

In [ ]:
len(titanic['fare'])

titanic.shape

### 2.2 Histogram
More informative in this case would be to look at the distribution over fares. We can visualise the distribution by plotting a histogram.

In [ ]:
titanic.fare.hist(bins = 10) # try different numbers of bins 
plt.xlim(0,300)               # setting limit on x-axis
plt.ylim(0,350)               # setting limit on y-axis

**Practice 4**: Reduce the x-axis limit to see how much most people paid to go on the titanic. Approximately how many people paid 10 or less? 

**Solution**:

In [ ]:
titanic.fare.hist(bins = 10) # try different numbers of bins 
plt.xlim(0,300)               # setting limit on x-axis
plt.ylim(0,350)               # setting limit on y-axis

(x-axis shows the fare, y-axis shows the frequency)

Approximately 330-340 passengers paid 10 or less

### 2.3 Boxplot

Alternatively, we can use a boxplot (also called a box and whisker diagram) to visualise the same data. A boxplot is a simple visual representation of key features of a univariate sample. It displays five-point summaries and potential outliers in graphical form. To create a boxplot we call:

In [ ]:
titanic.boxplot(column = 'fare')
plt.ylim(0, 150) # setting limit on y-axis

The red line across the centre of the box indicates the median value, i.e. half the data lies below the red line and half lies above it. The box itself defines the quartiles -- one quarter of the data lies above the box, and another quarter below it. We can see many high 'fare' values to the top of the graph. One might assume they are outliers, but it probably makes more sense to first investigate the different classes. We can generate boxplots divided by class, as follows:

In [ ]:
titanic.boxplot(column = 'fare', by = 'class')
plt.ylim(0, 600)

If we wanted to, we could filter out the large values in the different classes. For example, to filter out values greater than 160 in first class:

In [ ]:
filt = ~((titanic['class'] == 'First') & (titanic['fare'] > 160))
titanic = titanic[filt]
titanic.boxplot(column = 'fare', by = 'class')
plt.ylim(0, 600)

**Practice 5**: Use the same technique to filter out values greater than 50 for the second class and 30 for the third class. Plot the boxplot, and observe the graph. What is the median price for each class? Hint: set a lower y-axis limit to see clearer. 

**Solution**:

In [ ]:
# Filter out fares greater than 50 for the second class
filt2 = ~((titanic['class'] == 'Second') & (titanic['fare'] > 50))
titanic = titanic[filt2]

# Filter out fares greater than 30 for the third class
filt3 = ~((titanic['class'] == 'Third') & (titanic['fare'] > 30))
titanic = titanic[filt3]

In [ ]:
titanic.boxplot(column = 'fare', by = 'class')
plt.ylim(0, 20)

Median by class:
1. First Class - 55
2. Second Class - 14
3. Third Class - 8

### 2.4 Bar Chart
We can compare the fare for different classes and for children/adults using a bar chart.

**Practice 6a**: Fill in the missing code to make the aggregation function below count the number of children (age under 18) and adults (age 18 or over) in the different classes

**Solution**:


In [ ]:
fun_child_adult = {'age':{lambda x: sum(e<18 for e in x), lambda x: sum(e>=18 for e in x)}}

**Practice 6b**: Now follow the steps from Section 1.2 to group the 'titanic' data by class, and apply the above aggregation function to it. Call the resulting DataFrame 'groupbyClass2' and display it:

**Solution**:

In [ ]:
groupbyClass2 = titanic.groupby('class').agg(fun_child_adult)
groupbyClass2

In [ ]:
groupbyClass2.rename(
    columns = {
        '<lambda_0>':'child count',
        '<lambda_1>':'adult count'
    }, 
    inplace = True)
groupbyClass2

In [ ]:
# Old stuff
groupbyClass2 = groupbyClass2.reset_index()                  # turn 'class' groups into column values
groupbyClass2.columns = groupbyClass2.columns.droplevel(0)   # drop the top level in the column hierarchy
groupbyClass2.rename(columns = {'':'class'},inplace = True)
groupbyClass2

We'll now display the aggregated counts as a bar chart.

In [ ]:
ax=groupbyClass2.plot.bar(figsize=(8,5))# figsize sets size of plot
ax.set_xticklabels(groupbyClass2['class'],rotation=45)# use values of column 'class' as the x axis labels. Remove this line of code to see what will happen if we do not have this line
plt.xlabel('Ticket Class')# setting a label for x axis
plt.ylabel('Number of Passengers(child or adult)')# Setting a label for y axis
plt.title('Passengers ticket class based on their adulthood')# Setting the title of chart

**Practice 7**: So which class had the most families do you think?

**Solution**: Third class


### 2.6 Pie Chart:

**Practice 8**: Use the groupbyClass2 below to plot a pie chart, showing the number of children in each passenger class ('child count' column).

**Solution**

In [ ]:
groupbyClass2['child count']

In [ ]:
plt.pie(groupbyClass2['child count'])
#groupbyClass2.plot.pie(y=['child count'])
plt.show()

### 2.7 Scatter Plot

Input a simple data frame.

In [ ]:
df = pd.DataFrame({'Name' : ['Mike','Aaron','Brad','Steve','George','Mitchell','Shaun','Glenn','Pat','Robert','David'],
'Age' : [39,28,44,25,32,33,31,26,22,25,28],
'Runs' :[1310,662,1403,828,672,1140,655,1040,557,1030,1140]})
df

Let's have a quick look at the data by plotting it using an x-y scatter plot:

In [ ]:
plt.scatter(df['Age'], df['Runs'])
plt.show()

**Practice 9**: We now have two views of the same data, the table (DataFrame) view and the plot. What information do you gain/lose in these different views?

**Solution**:


## Basic plots

In [ ]:
plt.scatter(df['Age'], df['Runs'])
plt.show()

In [ ]:
plt.plot(df['Age'])
plt.show()

Histograms

In [ ]:
df.Age.hist(bins=2)
plt.show()

Boxplots

In [ ]:
df.boxplot(column='Age')
plt.show()

Bars

In [ ]:
plt.bar((1,2,3,4,5,6,7,8,9,10,11),df['Runs'])
plt.show()